# 古い形式のスケジュール設定を移行する
---

[2024/12/23](https://github.com/nii-gakunin-cloud/ocs-templates/releases/tag/release%2F23.11.0-20241223)以前のテンプレートにより設定された古い形式のスケジュール設定を新しい形式に移行する。

## パラメータの指定

新しいスケジュール定義に移行するために必要となるパラメータを指定します。

### VCCアクセストークンの入力

VCノードを操作するためにVC Controller(VCC)のアクセストークンが必要となります。
次のセルを実行すると表示される入力枠にVCCのアクセストークンを入力してください。

> アクセストークン入力後に Enter キーを押すことで入力が完了します。

In [ ]:
from getpass import getpass
vcc_access_token = getpass()

入力されたアクセストークンが正しいことを、実際にVCCにアクセスして確認します。

In [ ]:
from common import logsetting
from vcpsdk.vcpsdk import VcpSDK

vcp = VcpSDK(vcc_access_token)

上のセルの実行結果がエラーとなり以下のようなメッセージが表示されている場合は、入力されたアクセストークンに誤りがあります。

```
config vc failed: http_status(403)
2023/XX/XX XX:XX:XX UTC: VCPAuthException: xxxxxxx:token lookup is failed: permission denied
```

エラーになった場合はこの節のセルを全て `unfreeze` してから、もう一度アクセストークンの入力を行ってください。

### Ansibleのグループ名

対象となるスケジュール設定を行なったAnsibleのグループ名を指定します。

グループ名を確認するために `group_vars` ファイル名の一覧を表示します。

In [ ]:
!ls -1 --hide all group_vars/

Ansibleのグループ名を指定してください。

In [ ]:
# (例)
# target_group = 'scaleout'

target_group = 

#### チェック

指定されたグループ名が妥当なものであることをチェックします。

指定されたグループ名でスケジュール設定が行われ、かつ古い形式のスケジュール定義であることを確認します。

In [ ]:
%run scripts/group.py
gvars = load_group_vars(target_group)
if (old_cfg := gvars.get("vcnode_schedule")) is None:
    raise RuntimeError("スケジュールの設定が行われていません")
if not isinstance(old_cfg, list):
    raise RuntimeError("古い形式のスケジュール定義ではありません")

ugroup_name = gvars.get("ugroup_name")
if ugroup_name == target_group:
    target_group = f'{ugroup_name}_manager'
    group_name = ugroup_name
else:
    group_name = target_group

スケジュール設定を行なっているノードにアクセスできることを確認します。

In [ ]:
!ansible {target_group} -m ping

managerノードにアクセスできることを確認します。

In [ ]:
!ansible {ugroup_name}_manager -m ping
!ansible {ugroup_name}_manager -b -a 'whoami'

## 設定状況の確認

現在設定されている状態を確認します。

スケジュールの内容を表示するための環境を準備します。

In [ ]:
from pathlib import Path
!ansible-playbook -v -e venv_dir={str(Path.cwd() / ".venv")} playbooks/setup-local-venv.yml

現在設定されているスケジュールの内容を表示します。

In [ ]:
%run scripts/schedule.py
description_swarm_schedule(group_name)

## 新しいスケジュール定義形式に移行する

### 自動的な移行処理を利用する

現在の定義内容を、機械的な処理により自動的に新しいスケジュール定義形式に移行します。

現在の定義内容を新しい形式に変換できるかチェックします。次のセルを実行してエラーとならないことを確認してください。エラーとなる場合は「[3.2 手作業で移行を行う](#手作業で移行を行う)」で新しいスケジュール定義形式のパラメータを指定してください。

In [ ]:
%run scripts/schedule.py
ocs_vcnode_schedule = convert_to_ocs(group_name)

新しいスケジュール定義形式に移行した内容でgroup_varsを更新します。

In [ ]:
%run scripts/group.py
if ocs_vcnode_schedule is not None:
    ug_gvars = load_group_vars(ugroup_name)
    !cp group_vars/{ugroup_name} group_vars/.{ugroup_name}.bak || true
    update_group_vars(ugroup_name, vcnode_schedule=ocs_vcnode_schedule)
    if ug_gvars.get("schedule") == "power-ctl":
        ug = vcp.get_ugroup(ugroup_name)
        ut = ug.get_unit("worker")
        update_group_vars(ugroup_name, vcnode_all_ipaddress=ut.find_ip_addresses())
    else:
        update_group_vars(ugroup_name, vcnode_all_ipaddress=gvars.get("vcnode_all_ipaddress"))
    remove_group_vars(ugroup_name, "schedule", if_exists=True)
    remove_group_vars(ugroup_name, "vcnode_ctl_dir", if_exists=True)
    
    if not group_name.startswith(ugroup_name):
        !cp group_vars/{group_name} group_vars/.{group_name}.bak || true
        remove_group_vars(group_name, "vcnode_schedule", if_exists=True)

group_varsの差分を確認します。

In [ ]:
! ! diff -u group_vars/.{ugroup_name}.bak group_vars/{ugroup_name}

変換後のスケジュール定義の内容を表示します。「[2. 設定状況の確認](#設定状況の確認)」で表示したものと同じ内容であることを確認してください。

In [ ]:
%run scripts/schedule.py
description_schedule(ugroup_name)

### 手作業で移行を行う

機械的な変換処理が利用できない場合は、新しい定義形式の各パラメータを指定する必要があります。

> 「[3.1 自動的な移行処理を利用する](#自動的な移行処理を利用する)」でgroup_varsを更新した場合は、この節を実行する必要はありません。

以下に記した内容に沿って、新たなスケジュール定義形式のパラメータを指定してください。


* [3.2.1 スケジュールの指定](#スケジュールの指定)
* [3.2.2 通常時のノード数](#通常時のノード数)
* [3.2.3 新たなタスクの受付を停止する期間の指定](#新たなタスクの受付を停止する期間の指定)

#### スケジュールの指定

workerノード数を変更する期間とそのノード数を指定します。

ひとつのスケジュールを定義するには、期間を`begin`(開始時刻)と`end`(終了時刻)で指定し、その期間におけるworkerノード数を`node_count`で指定します。この３つの値をpythonのdictで指定します。例えば毎日22:00から翌朝7:00まではworkerノード数を1にするスケジュールは以下のような値で定義します。

```python
{
    "begin": "22:00",
    "end": "7:00",
    "node_count": 1,
}
```

期間は日毎、週毎、月毎、年毎、特定日時の指定が行えます。

日毎の場合は`22:00`のように`{時}:{分}`の形式で指定します。

週毎の場合は`Sat 0:00`のように`{曜日} {時}:{分}`の形式で指定します。曜日は`Sunday`、`日曜日`、`Sun`、`日`のような値で指定を行います。

月毎の場合は`1日 0:00`のように`{日} {時}:{分}`の形式で指定します。日は`1日`、`1`のような値で指定を行います。

年毎の場合は`12/29 0:00`のように`{月}/{日} {時}:{分}`の形式で指定します。

特定日時の場合は`2025/4/1 0:00`のように`{年}/{月}/{日} {時}:{分}`の形式で指定します。

期間の開始と終了は同じ粒度（日毎、週毎、月毎、年毎、特定日時）を指定する必要があります。

次のセルでスケジュール定義のリストを指定してください。リストの個々の要素は上で説明した`begin`, `end`, `node_count`をキーに持つdictになります。複数のスケジュール期間が重複した場合、リストで先に指定したものが優先されます。

In [ ]:
schedule_list = [
# (例)
#    {"begin": "9:00", "end": "16:00", "node_count": 5},                                        # 日毎のスケジュール 9:00--16:00
#    {"begin": "Mon 9:00", "end": "Fri 20:00", "node_count": 5},                          # 週毎のスケジュール 月曜日9:00--金曜日20:00
#    {"begin": "1日 0:00", "end": "1日 12:00", "node_count": 5},                         # 月毎のスケジュール 1日0:00--1日12:00
#    {"begin": "12/29 0:00", "end": "1/4 0:00", "node_count": 0},                     # 年毎のスケジュール 12月29日0:00--1月4日0:00
#    {"begin": "2025/4/1 0:00", "end": "2025/4/2 0:00", "node_count": 5},   # 特定日時のスケジュール 2025/4月1日0:00--2025/4月2日0:00

]

リスト`schedule_list`に指定したスケジュールをチェックします。次のセルを実行してエラーにならないことを確認してください。指定されたスケジュールのリストに問題ない場合は設定した内容を表示します。意図したスケジュールが指定できていることを確認してください。

In [ ]:
import sys
ug_vars = load_group_vars(ugroup_name)
schedule_down_type = "power_down" if ug_vars.get("schedule") == "power-ctl" else "deleted"
params = {name: value for name, value in vars().items() if name.startswith("schedule")}
try:
    description_schedule(ugroup_name, params, "node_count")
except ValueError as e:
    print(f"ERROR: {e}", file=sys.stderr)
    raise

#### 通常時のノード数

通常時のworkerノード数を次のセルで指定してください。

In [ ]:
# (例)
# schedule_default_nodes = 1

schedule_default_nodes = 

#### 新たなタスクの受付を停止する期間の指定

ノードを停止する前に、タスクの受付を停止し新たなコンテナを起動できないようにする期間を設けます。これは、Docker Swarmに対してノードのAvailabilityを`Pause`にすることで実現します。Availabilityを`Pause`にする時間を次のセルで指定してください。指定の単位は分となります。

In [ ]:
# (例)
# schedule_drain_time = 60

schedule_drain_time = 

#### 設定内容の確認

ここまで設定した内容を確認します。

In [ ]:
params = {name: value for name, value in vars().items() if name.startswith("schedule")}
description_schedule(ugroup_name, params)

#### group_varsの更新

In [ ]:
%run scripts/schedule.py
ug_gvars = load_group_vars(ugroup_name)
!cp group_vars/{ugroup_name} group_vars/.{ugroup_name}.bak || true

params = {name: value for name, value in vars().items() if name.startswith("schedule")}
update_group_vars(ugroup_name, vcnode_schedule=get_schedule_definition(ugroup_name, params))
if ug_gvars.get("schedule") == "power-ctl":
    ug = vcp.get_ugroup(ugroup_name)
    ut = ug.get_unit("worker")
    update_group_vars(ugroup_name, vcnode_all_ipaddress=ut.find_ip_addresses())
else:
    update_group_vars(ugroup_name, vcnode_all_ipaddress=gvars.get("vcnode_all_ipaddress"))
remove_group_vars(ugroup_name, "schedule", if_exists=True)
remove_group_vars(ugroup_name, "vcnode_ctl_dir", if_exists=True)

if not group_name.startswith(ugroup_name):
    !cp group_vars/{group_name} group_vars/.{group_name}.bak || true
    remove_group_vars(group_name, "vcnode_schedule", if_exists=True)

## 配備

新しい形式に変換したスケジュール定義を用いてスケジュール設定に関する資材をmanagerノードに配備します。

次のセルを実行すると、現在実行中のsystemdタイマーの設定を無効化し、その後に新たな資材の配備とsystemdタイマーの開始を行います。

In [ ]:
import json
from pathlib import Path
from tempfile import TemporaryDirectory

with TemporaryDirectory() as work_dir:
    vars_path = Path(work_dir) / "params.json"
    with vars_path.open(mode="w") as f:
        json.dump({"vcc_access_token": vcc_access_token}, f)
    !ansible-playbook -Dv -l {target_group} playbooks/disable-swarm-schedule.yml
    !ansible-playbook -Dv -l {ugroup_name}_manager -e @{str(vars_path)} playbooks/setup-schedule.yml

CoursewareHubのmanagerノードに配備した資材を確認します。

In [ ]:
!ansible {ugroup_name}_manager -b -a 'tree chdir=/srv/worker-schedule'

In [ ]:
!ansible {ugroup_name}_manager -b -m shell -a 'ls -l cwh* chdir=/etc/systemd/system/'

開始したsystemdタイマーの状態を確認します。通常は１５分ごとにworkerノード状態の確認とノード数の更新を行います。

In [ ]:
!ansible {ugroup_name}_manager -b -a 'systemctl list-timers cwh-worker-node.timer'

不要となったAnsibleのグループを整理します。

In [ ]:
if not group_name.startswith(ugroup_name):
    !mv group_vars/{target_group} group_vars/.{target_group}.old
    remove_group_from_inventory_yml(target_group)